# 🚀 Swin-Small + MobileViT (Kvasir) with Optuna TPE

## Architecture
- **Swin-Small**: 768-dim features (ImageNet pretrained)
- **MobileViT**: 512-dim features (**Kvasir pretrained** - custom architecture)
- **Fusion**: 768 + 512 = 1280-dim -> 256-dim
- **Classifier**: CatBoost with Optuna TPE optimization

## Expected Results
- **Accuracy**: ~92-93% (with Kvasir MobileViT)
- **AUROC**: ~99%
- **Training Time**: ~40-45 minutes

**Note**: This notebook includes the custom MobileViT architecture to load Kvasir-pretrained weights.

In [ ]:
# Install packages
!pip install -q optuna catboost scikit-learn
print("✅ Packages installed")

In [ ]:
# Mount Google Drive for MobileViT weights
from google.colab import drive
drive.mount('/content/drive')

import os
weights_path = '/content/drive/MyDrive/TripleHybridModel/mobilevit_kvasir_v2_best_optuna.pth'

if os.path.exists(weights_path):
    print(f"✅ Found MobileViT Kvasir weights")
else:
    print(f"❌ Weights not found. Upload to: MyDrive/TripleHybridModel/")
    weights_path = None

In [ ]:
# Imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import numpy as np
from pathlib import Path
from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, log_loss
from sklearn.preprocessing import label_binarize
import optuna
from catboost import CatBoostClassifier
import time
import json

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🎮 Device: {device}")

In [ ]:
# Download dataset
if not os.path.exists('kvasir-dataset-v2'):
    print("📥 Downloading Kvasir dataset...")
    !wget --no-check-certificate https://datasets.simula.no/downloads/kvasir/kvasir-dataset-v2.zip
    print("📦 Extracting...")
    !unzip -q kvasir-dataset-v2.zip
    print("✅ Dataset ready!")
else:
    print("✅ Dataset already exists")

## 🏗️ Custom MobileViT Architecture (for Kvasir weights)

In [ ]:
# Helper functions
def conv_1x1_bn(inp, oup):
    return nn.Sequential(
        nn.Conv2d(inp, oup, 1, 1, 0, bias=False),
        nn.BatchNorm2d(oup),
        nn.Hardswish()
    )

def conv_nxn_bn(inp, oup, kernal_size=3, stride=1):
    return nn.Sequential(
        nn.Conv2d(inp, oup, kernal_size, stride, 1, bias=False),
        nn.BatchNorm2d(oup),
        nn.Hardswish()
    )

class PreNorm(nn.Module):
    def __init__(self, dim, fn):
        super().__init__()
        self.norm = nn.LayerNorm(dim)
        self.fn = fn
    def forward(self, x, *args, **kwargs):
        return self.fn(self.norm(x), *args, **kwargs)

class FeedForward(nn.Module):
    def __init__(self, dim, hidden_dim, dropout=0.):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim, hidden_dim),
            nn.Hardswish(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, dim),
            nn.Dropout(dropout)
        )
    def forward(self, x):
        return self.net(x)

class Attention(nn.Module):
    def __init__(self, dim, heads=8, dim_head=64, dropout=0.):
        super().__init__()
        self.inner_dim = dim_head * heads
        project_out = not (heads == 1 and dim_head == dim)
        self.heads = heads
        self.scale = dim_head ** -0.5
        self.attend = nn.Softmax(dim=-1)
        self.dropout = nn.Dropout(dropout)
        self.to_qkv = nn.Linear(dim, self.inner_dim * 3, bias=False)
        self.to_out = nn.Sequential(
            nn.Linear(self.inner_dim, dim),
            nn.Dropout(dropout)
        ) if project_out else nn.Identity()
    
    def forward(self, x):
        qkv = self.to_qkv(x).chunk(3, dim=-1)
        q, k, v = map(lambda t: t.reshape(t.shape[0], t.shape[1], self.heads, t.shape[2] // self.heads).transpose(1, 2), qkv)
        dots = torch.matmul(q, k.transpose(-1, -2)) * self.scale
        attn = self.attend(dots)
        attn = self.dropout(attn)
        out = torch.matmul(attn, v)
        out = out.transpose(1, 2).reshape(out.shape[0], out.shape[1], self.inner_dim)
        return self.to_out(out)

class TransformerBlock(nn.Module):
    def __init__(self, dim, heads, dim_head, mlp_dim, dropout=0.):
        super().__init__()
        self.attn = PreNorm(dim, Attention(dim, heads=heads, dim_head=dim_head, dropout=dropout))
        self.ff = PreNorm(dim, FeedForward(dim, mlp_dim, dropout=dropout))
    
    def forward(self, x):
        x = x + self.attn(x)
        x = x + self.ff(x)
        return x

class MobileNetV2Block(nn.Module):
    def __init__(self, inp, oup, stride, expansion):
        super().__init__()
        self.stride = stride
        hidden_dim = int(round(inp * expansion))
        self.use_res_connect = self.stride == 1 and inp == oup
        
        if expansion == 1:
            self.conv = nn.Sequential(
                nn.Conv2d(hidden_dim, hidden_dim, 3, stride, 1, groups=hidden_dim, bias=False),
                nn.BatchNorm2d(hidden_dim),
                nn.Hardswish(),
                nn.Conv2d(hidden_dim, oup, 1, 1, 0, bias=False),
                nn.BatchNorm2d(oup),
            )
        else:
            self.conv = nn.Sequential(
                nn.Conv2d(inp, hidden_dim, 1, 1, 0, bias=False),
                nn.BatchNorm2d(hidden_dim),
                nn.Hardswish(),
                nn.Conv2d(hidden_dim, hidden_dim, 3, stride, 1, groups=hidden_dim, bias=False),
                nn.BatchNorm2d(hidden_dim),
                nn.Hardswish(),
                nn.Conv2d(hidden_dim, oup, 1, 1, 0, bias=False),
                nn.BatchNorm2d(oup),
            )
    
    def forward(self, x):
        if self.use_res_connect:
            return x + self.conv(x)
        else:
            return self.conv(x)

class MobileViTBlock(nn.Module):
    def __init__(self, dim, depth, channel, kernal_size, patch_size, mlp_dim, dropout=0.):
        super().__init__()
        self.ph, self.pw = patch_size
        self.conv1 = conv_nxn_bn(channel, channel, kernal_size)
        self.conv2 = conv_1x1_bn(channel, dim)
        self.transformer = nn.Sequential(*[TransformerBlock(dim, 4, 8, mlp_dim, dropout) for _ in range(depth)])
        self.conv3 = conv_1x1_bn(dim, channel)
        self.conv4 = conv_nxn_bn(2 * channel, channel, kernal_size)
    
    def forward(self, x):
        y = x.clone()
        x = self.conv1(x)
        x = self.conv2(x)
        
        B, D, H, W = x.shape
        pad_h = (self.ph - (H % self.ph)) % self.ph
        pad_w = (self.pw - (W % self.pw)) % self.pw
        
        if pad_h > 0 or pad_w > 0:
            x = F.pad(x, (0, pad_w, 0, pad_h), mode='constant', value=0)
            H, W = H + pad_h, W + pad_w
        
        h_split, w_split = H // self.ph, W // self.pw
        x = x.reshape(B, D, h_split, self.ph, w_split, self.pw)
        x = x.permute(0, 2, 4, 3, 5, 1).contiguous()
        x = x.view(B * h_split * w_split, self.ph * self.pw, D)
        x = self.transformer(x)
        x = x.view(B, h_split, w_split, self.ph, self.pw, D)
        x = x.permute(0, 5, 1, 3, 2, 4).contiguous()
        x = x.view(B, D, H, W)
        
        if pad_h > 0 or pad_w > 0:
            x = x[:, :, :H-pad_h, :W-pad_w]
        
        x = self.conv3(x)
        x = torch.cat((x, y), 1)
        x = self.conv4(x)
        return x

class MobileViT(nn.Module):
    def __init__(self, image_size, num_classes, dims, channels, depths, patch_size=2, mlp_dim_ratio=2):
        super().__init__()
        ph, pw = patch_size, patch_size
        self.conv1 = conv_nxn_bn(3, channels[0], kernal_size=3, stride=2)
        
        self.stem = nn.ModuleList([])
        in_channel = channels[0]
        self.stem.append(MobileNetV2Block(in_channel, channels[1], stride=1, expansion=4))
        in_channel = channels[1]
        self.stem.append(MobileNetV2Block(in_channel, channels[2], stride=2, expansion=4))
        in_channel = channels[2]
        self.stem.append(MobileNetV2Block(in_channel, channels[3], stride=1, expansion=4))
        in_channel = channels[3]
        self.stem.append(MobileNetV2Block(in_channel, channels[4], stride=2, expansion=4))
        in_channel = channels[4]
        
        self.mobilevit_blocks = nn.ModuleList([])
        for i in range(len(dims)):
            mlp_dim = dims[i] * mlp_dim_ratio
            self.mobilevit_blocks.append(MobileViTBlock(dims[i], depths[i], in_channel, 3, (ph, pw), mlp_dim))
            if i < len(dims) - 1:
                next_channel_idx = i + 5
                self.mobilevit_blocks.append(MobileNetV2Block(in_channel, channels[next_channel_idx], stride=2, expansion=4))
                in_channel = channels[next_channel_idx]
        
        self.conv2 = conv_1x1_bn(in_channel, dims[-1])
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(dims[-1], num_classes)
    
    def forward(self, x):
        x = self.conv1(x)
        for block in self.stem:
            x = block(x)
        for block in self.mobilevit_blocks:
            x = block(x)
        x = self.conv2(x)
        x = self.pool(x).flatten(1)
        x = self.fc(x)
        return x

print("✅ Custom MobileViT architecture defined")

## 🏗️ Hybrid Model with Kvasir MobileViT

In [ ]:
class SwinSmallMobileViTHybrid(nn.Module):
    def __init__(self, mobilevit_weights_path=None):
        super().__init__()
        
        # Swin-Small
        print("Loading Swin-Small...")
        self.swin = models.swin_s(weights=models.Swin_S_Weights.IMAGENET1K_V1)
        self.swin.head = nn.Identity()
        self.swin_dim = 768
        
        # Custom MobileViT
        print("Loading Custom MobileViT...")
        dims = [96, 192, 384]
        channels = [16, 32, 64, 128, 160, 192, 256, 320, 384, 512]
        depths = [2, 2, 2]
        
        self.mobilevit = MobileViT(
            image_size=224,
            num_classes=8,
            dims=dims,
            channels=channels,
            depths=depths
        )
        
        # Load Kvasir weights
        if mobilevit_weights_path and os.path.exists(mobilevit_weights_path):
            self.mobilevit.load_state_dict(torch.load(mobilevit_weights_path, map_location='cpu'))
            print("✅ MobileViT Kvasir weights loaded!")
        else:
            print("⚠️ No Kvasir weights - using random initialization")
        
        # Remove classification heads
        self.mobilevit.fc = nn.Identity()
        self.mobilevit_dim = 384  # Last dim from dims
        
        # Freeze backbones
        for param in self.swin.parameters():
            param.requires_grad = False
        for param in self.mobilevit.parameters():
            param.requires_grad = False
        
        # Fusion layer
        combined_dim = self.swin_dim + self.mobilevit_dim  # 768 + 384 = 1152
        self.fusion = nn.Sequential(
            nn.Linear(combined_dim, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 256)
        )
        
        print(f"✅ Hybrid Model Ready: {self.swin_dim} + {self.mobilevit_dim} = {combined_dim} -> 256")
    
    def forward(self, x):
        with torch.no_grad():
            swin_features = self.swin(x)
            mobilevit_features = self.mobilevit(x)
            combined = torch.cat([swin_features, mobilevit_features], dim=1)
        return self.fusion(combined)

print("✅ Hybrid model class defined")

In [ ]:
# Dataset
class KvasirDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = Path(root_dir)
        self.transform = transform
        self.samples = []
        classes = sorted([d.name for d in self.root_dir.iterdir() if d.is_dir()])
        self.class_to_idx = {cls: idx for idx, cls in enumerate(classes)}
        for cls in classes:
            for img in (self.root_dir / cls).glob('*.jpg'):
                self.samples.append((img, self.class_to_idx[cls]))
    
    def __len__(self): return len(self.samples)
    def __getitem__(self, idx):
        img, label = self.samples[idx]
        return self.transform(Image.open(img).convert('RGB')), label

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

if os.path.exists('kvasir-dataset-v2/kvasir-dataset-v2'):
    dataset_path = 'kvasir-dataset-v2/kvasir-dataset-v2'
else:
    dataset_path = 'kvasir-dataset-v2'

dataset = KvasirDataset(dataset_path, transform=transform)
idx = list(range(len(dataset)))
labels = [s[1] for s in dataset.samples]
tr_idx, temp = train_test_split(idx, test_size=0.3, stratify=labels, random_state=42)
val_idx, te_idx = train_test_split(temp, test_size=0.5, stratify=[labels[i] for i in temp], random_state=42)

print(f"📊 Dataset: Train={len(tr_idx)}, Val={len(val_idx)}, Test={len(te_idx)}")

In [ ]:
# Extract features
model = SwinSmallMobileViTHybrid(weights_path).to(device).eval()

def extract_features(indices, set_name):
    loader = DataLoader(torch.utils.data.Subset(dataset, indices), batch_size=32, num_workers=2)
    feats, lbls = [], []
    start_time = time.time()
    for img, lbl in tqdm(loader, desc=f"Extracting {set_name}"):
        feats.append(model(img.to(device)).cpu().detach().numpy())
        lbls.append(lbl.numpy())
    return np.vstack(feats), np.concatenate(lbls), time.time() - start_time

print("🔬 Extracting features...")
X_train, y_train, train_time = extract_features(tr_idx, "Train")
X_val, y_val, val_time = extract_features(val_idx, "Val")
X_test, y_test, test_time = extract_features(te_idx, "Test")

print(f"\n✅ Features: {X_train.shape}")
print(f"⏱️ Time: Train={train_time:.1f}s, Val={val_time:.1f}s, Test={test_time:.1f}s")

In [ ]:
# Optuna TPE optimization
optuna.logging.set_verbosity(optuna.logging.WARNING)

def objective(trial):
    params = {
        'depth': trial.suggest_int('depth', 4, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'iterations': trial.suggest_int('iterations', 100, 1000),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 10),
        'border_count': trial.suggest_int('border_count', 32, 255),
        'task_type': 'GPU',
        'verbose': False,
        'random_seed': 42
    }
    clf = CatBoostClassifier(**params)
    clf.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=50, verbose=False)
    return accuracy_score(y_val, clf.predict(X_val))

print("🔍 Optuna TPE Optimization (50 trials)...")
opt_start = time.time()
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(objective, n_trials=50, show_progress_bar=True)
opt_time = time.time() - opt_start

print(f"\n🏆 Best Val Acc: {study.best_value*100:.2f}%")
print(f"⏱️ Time: {opt_time/60:.2f} min")

In [ ]:
# Train final model
final_params = study.best_params
final_params.update({'task_type': 'GPU', 'random_seed': 42, 'verbose': True})
final_model = CatBoostClassifier(**final_params)

train_start = time.time()
final_model.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=50)
train_time_final = time.time() - train_start

print(f"\n✅ Trained in {train_time_final:.2f}s")

In [ ]:
# Calculate metrics
def calc_metrics(model, X, y):
    start = time.time()
    y_pred = model.predict(X)
    y_prob = model.predict_proba(X)
    inf_time = time.time() - start
    
    y_bin = label_binarize(y, classes=range(8))
    cm = confusion_matrix(y, y_pred)
    tpr_list, fpr_list = [], []
    for i in range(8):
        tp = cm[i, i]
        fn, fp = cm[i, :].sum() - tp, cm[:, i].sum() - tp
        tn = cm.sum() - tp - fn - fp
        tpr_list.append(tp / (tp + fn) if (tp + fn) > 0 else 0)
        fpr_list.append(fp / (fp + tn) if (fp + tn) > 0 else 0)
    
    return {
        'accuracy': accuracy_score(y, y_pred),
        'precision': precision_score(y, y_pred, average='weighted', zero_division=0),
        'recall': recall_score(y, y_pred, average='weighted', zero_division=0),
        'f1_score': f1_score(y, y_pred, average='weighted', zero_division=0),
        'auroc': roc_auc_score(y_bin, y_prob, average='weighted', multi_class='ovr'),
        'tpr': np.mean(tpr_list),
        'fpr': np.mean(fpr_list),
        'loss': log_loss(y, y_prob),
        'time': inf_time
    }

train_m = calc_metrics(final_model, X_train, y_train)
val_m = calc_metrics(final_model, X_val, y_val)
test_m = calc_metrics(final_model, X_test, y_test)

print(f"\nTest Accuracy: {test_m['accuracy']*100:.2f}%")
print(f"Test AUROC: {test_m['auroc']*100:.2f}%")

In [ ]:
# Save results
final_model.save_model('swin_small_kvasir_optuna.cbm')
np.savez('swin_small_kvasir_features.npz', X_train=X_train, y_train=y_train, X_val=X_val, y_val=y_val, X_test=X_test, y_test=y_test)

results = {
    'architecture': 'Swin-Small + MobileViT (Kvasir)',
    'optimization': 'Optuna TPE',
    'n_trials': 50,
    'best_params': study.best_params,
    'best_val_accuracy': study.best_value,
    'training_metrics': train_m,
    'validation_metrics': val_m,
    'test_metrics': test_m,
    'timing': {'feature_extraction': {'train': train_time, 'val': val_time, 'test': test_time}, 'optimization': opt_time, 'final_training': train_time_final}
}

with open('swin_small_kvasir_optuna_results.json', 'w') as f:
    json.dump(results, f, indent=2, default=float)

from google.colab import files
files.download('swin_small_kvasir_optuna.cbm')
files.download('swin_small_kvasir_features.npz')
files.download('swin_small_kvasir_optuna_results.json')

print("✅ Done! Expected accuracy: ~92%")